# Faster R-CNN

Faster R-CNN[1]の実装を動かします。


[1]Ren, S., He, K., Girshick, R., Sun, J.: Faster r-cnn: Towards real-time object detection with region proposal networks. In: Advances in neural information processing systems. (2015) 91–99


https://arxiv.org/pdf/1506.01497.pdf


以下のものを使用してください。Kerasを使用した実装です。


[duckrabbits/ObjectDetection at master](https://github.com/duckrabbits/ObjectDetection/tree/master)

##【問題1】学習と推定
READMEを参考に上記実装を動かしてください。

In [2]:
import os
import tensorflow
tensorflow.VERSION
# !pip install kaggle

'1.15.0'

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
!mkdir .kaggle
!ls -a

mkdir: cannot create directory ‘.kaggle’: File exists
.  ..  .config	drive  .kaggle	sample_data


In [0]:
# https://www.kaggle.com/　へアクセスし、ログイン後に右上のプロフィール画像をクリック、さらに"My Account"をクリックする
# このサイトの「API」項目にある"Create New API Token"をクリックすると、kaggle.jsonファイルが自動的にダウンロードされる
# ローカルで、kaggle.json（ダウンロードフォルダにあるはず）をエディターで開く。
# このセルの以下のコードにある token = {'username':'***','key':'***'} における「***」部分を、
# ダウンロードしたkaggle.jsonを参照して書き換え、このセルを実行する

import json

token = {"username":"ryohanoi","key":"21565eed4a7fe5fa312665088bdf2d20"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [7]:
!chmod 600 /content/.kaggle/kaggle.json
!ls -a .kaggle
!mkdir　/root/.kaggle/
!cp -R /content/.kaggle/kaggle.json /root/.kaggle/
!ls -a /root/.kaggle/
# kaggle APIコマンドが使えるかどうかを確認
!kaggle competitions list

.  ..  kaggle.json
/bin/bash: mkdir　/root/.kaggle/: No such file or directory
cp: cannot create regular file '/root/.kaggle/': Not a directory
ls: cannot access '/root/.kaggle/': No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, self.config_dir))
IOError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [8]:
# カレントディレクトリの変更
os.chdir('/content/drive/My Drive/')

# カレントディレクトリの取得
print(os.getcwd())

/content/drive/My Drive


In [0]:
# simpsonデータセットのAPIコマンドでデータセットをダウンロードする
# APIコマンドはこちらのサイトを参照。https://www.kaggle.com/alexattia/the-simpsons-characters-dataset

# !kaggle datasets download -d alexattia/the-simpsons-characters-dataset
# !unzip /content/drive/'My Drive'/the-simpsons-characters-dataset.zip  -d Simpsons

In [0]:
# 埋め込み／上書き注意
# !git clone https://github.com/duckrabbits/ObjectDetection.git

In [0]:
# https://github.com/duckrabbits/ObjectDetection/tree/master
# この後、上述のgithubのreadmeに沿って、simpsons_datasetフォルダをプロジェクトフォルダ配下に格納しましょう。
# testデータなども同様に準備してみよう。

In [0]:
os.chdir('/content/drive/My Drive/ObjectDetection')

In [13]:
# 手動でシンプソンデータをObjectDetection に移動させよう
# 差し替え必要！
!python train.py -p annotation.txt # 実行前にsimpsons_datasetをObjectDetection直下に移動させる必要がある。

Using TensorFlow backend.
Parsing annotation files
suzuki
suzuki
suzuki
suzuki
suzuki
suzuki
katoh
kato
Training images per class (2 classes) :
{'bart_simpson': 6, 'bg': 0}
-------------------------------
path to config file : ./save/train_20200325-011253_config.pickle
-------------------------------









Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, None, None, 3 0           input_1[0][0]                    
___________________________________________________________________________________

## 【問題2】コードリーディング
上記実装のコードリーディングを行ってください。


まず、Faster R-CNN[1]において重要だと考えた部分を列挙してください。そして、それに対応するコードを見つけてください。


（例）


RPNを実現しているコードはどこか
RoIプーリングを実現しているコードはどこか

フレームワークには畳み込み層など一般的なものはクラスが用意されていますが、RoIプーリングなど特定の手法限定のものは用意されていません。オリジナルのレイヤーを作成することが可能であり、Kerasであれば以下のページに情報がまとまっています。


オリジナルのKerasレイヤーを作成する - Keras Documentation


《参考》


KerasではVGG16のクラスが用意されているため、簡単に利用ができます。include_top=Falseの引数を与えることで、出力のための全結合層部分が除かれます。weights='imagenet'でImageNetを利用した学習済みモデルも手に入り、転移学習 が行えます。weights='None'とすればランダムな初期化となります。


Applications - Keras Documentation

* RPNを実現しているコードはどこか
→ resnet.py

* RoIプーリングを実現しているコードはどこか
→ RoiPoolingConv.py

## 【問題3】学習済みの重みによる推定
学習済みの重みを使い推定を行う方法がREADME.mdのQuick Startの項に記載されています。


まずはこの通りにして各自何かしらの画像や動画に対して検出を行ってください。


出力結果を課題の一部として提出してください。

In [0]:
os.chdir('/content/drive/My Drive')
# !git clone https://github.com/qqwweee/keras-yolo3.git
os.chdir('/content/drive/My Drive/keras-yolo3')
# !wget https://pjreddie.com/media/files/yolov3.weights
# !python convert.py yolov3.cfg yolov3.weights model_data/yolo.h5

In [0]:
!python yolo_video.py --image

Using TensorFlow backend.
Image detection mode
 Ignoring remaining command line arguments: ./path2your_video,



2020-03-25 01:31:40.886081: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200000000 Hz
2020-03-25 01:31:40.886284: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x2450bc0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-03-25 01:31:40.886314: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2020-03-25 01:31:40.887989: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-03-25 01:31:41.001635: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-03-25 01:31:41.002862: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x24

## 【問題4】学習のためのファイルを作成
新しいデータ（シンプソンズデータセット）を学習します。README.mdのTrainingの項を読み、シンプソンズデータセットを学習するために必要なファイルを作成してください。


アノテーションファイルの形式が問題1の実装とは異なるため変換する必要があります。

In [0]:
# /content/drive/My Drive/Simpsons/kaggle_simpson_testset/kaggle_simpson_testset/abraham_grampa_simpson_0.jpg

In [0]:
# !python convert.py -w yolov3.cfg yolov3.weights model_data/yolo_weights.h5

## 【問題5】学習が行えることの確認
問題4で作成したファイルを使用して学習してください。実行環境で学習に時間がかかる場合は、学習が行えることを確認するのみで終えて構いません。

In [0]:
!python train.py

Using TensorFlow backend.








2020-03-25 01:33:55.316595: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200000000 Hz
2020-03-25 01:33:55.316802: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1626a00 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-03-25 01:33:55.316833: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2020-03-25 01:33:55.318715: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-03-25 01:33:55.429699: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-03-25 01:33:55.430384: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1626bc0 initialized for platform CUDA (this does not guarantee that XLA will be u